In [4]:
import cugrad

a = cugrad.Tensor(2.0)
b = cugrad.Tensor(-3.0)
c = cugrad.Tensor(10.0)

print("a =", a)  # <Tensor data=2.0, grad=0.0>
print("b =", b)  # <Tensor data=-3.0, grad=0.0>
print("c =", c)  # <Tensor data=10.0, grad=0.0>

print("\nPerforming arithmetic operations:")
d = a * b + c
print("d = a * b + c =", d)  # Expected: <Tensor data=4.0, grad=-3.0>


a = <Value data=2.000000, grad=0.000000>
b = <Value data=-3.000000, grad=0.000000>
c = <Value data=10.000000, grad=0.000000>

Performing arithmetic operations:
d = a * b + c = <Value data=4.000000, grad=0.000000>
